In [1]:
# import 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [2]:
# improt the data 
import pandas as pd

data = pd.read_csv('IMDB Dataset.csv')

In [3]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
# Download necessary NLTK corpora

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text, re.UNICODE) # remove punctuation
    text = text.lower() # convert to lowercase
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")] # lemmatization (hampir sama kek steming)
    text = [word for word in text if not word in stop_words] # remove stopwords
    text = " ".join(text)
    return text

# Apply the text cleaning function to our data
data['Processed_Reviews'] = data.review.apply(lambda x: clean_text(x))

In [5]:
data.head()

,review,sentiment,Processed_Reviews
0,One of the other reviewers has mentioned that ...,positive,one reviewer ha mentioned watching 1 oz episod...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production br br filming tech...
2,I thought this was a wonderful way to spend ti...,positive,thought wa wonderful way spend time hot summer...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [6]:
positf = 0
negatif = 0
for i in range(len(data)):
    sen = data['sentiment'][i]
    if sen == 'positive':
        positf = positf + 1
    else :
        negatif = negatif + 1


print("data positif : ",positf)
print("data negatif : ",negatif)

data positif :  25000
data negatif :  25000


In [7]:
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()

data['sentiment'] = encode.fit_transform(data['sentiment'])

### with ann

In [9]:
from nltk.tokenize import word_tokenize

def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['token'] = data['Processed_Reviews'].apply(word_tokenize_wrapper)
data

,review,sentiment,Processed_Reviews,token
0,One of the other reviewers has mentioned that ...,1,one reviewer ha mentioned watching 1 oz episod...,"[one, reviewer, ha, mentioned, watching, 1, oz..."
1,A wonderful little production. <br /><br />The...,1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,1,thought wa wonderful way spend time hot summer...,"[thought, wa, wonderful, way, spend, time, hot..."
3,Basically there's a family where a little boy ...,0,basically family little boy jake think zombie ...,"[basically, family, little, boy, jake, think, ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."
...,...,...,...,...
49995,I thought this movie did a down right good job...,1,thought movie right good job wasnt creative or...,"[thought, movie, right, good, job, wasnt, crea..."
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,bad plot bad dialogue bad acting idiotic direc...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,I am a Catholic taught in parochial elementary...,0,catholic taught parochial elementary school nu...,"[catholic, taught, parochial, elementary, scho..."
49998,I'm going to have to disagree with the previou...,0,im going disagree previous comment side maltin...,"[im, going, disagree, previous, comment, side,..."


In [30]:
import numpy as np
from keras.preprocessing.text import Tokenizer
# data_seq = np.array(data['token'])
# data_seq = data_seq.reshape(-1,1)

data_seq = list(data['token'])

# Use Tokenizer to convert words to numerical indices
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_seq)
data_seq_numeric = tokenizer.texts_to_sequences(data_seq)


In [24]:
# mencetak max kata dalam document
len_max = max(len(seq)for seq in data['token'])
print(len_max)

1826


In [32]:
from keras.preprocessing.sequence import pad_sequences
data_pad = pad_sequences(data_seq_numeric,maxlen=len_max)

In [33]:
test_x = data_pad[:10000]
test_y = data['sentiment'][:10000]
train_x = data_pad[10000:]
train_y = data['sentiment'][10000:]

In [34]:
print(test_x.shape,test_y.shape,train_x.shape,train_y.shape)

(10000, 1826) (10000,) (40000, 1826) (40000,)


In [42]:
import keras

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=(len(tokenizer.word_index)+1), output_dim=64, input_length=len_max),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics='acc'
)

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1826, 64)          11594880  
                                                                 
 lstm_2 (LSTM)               (None, 1826, 64)          33024     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_16 (Dense)            (None, 64)                2112      
                                                                 
 dense_17 (Dense)            (None, 32)                2080      
                                                                 
 dense_18 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11644545 (44.42 MB)
Trainable params: 11

In [43]:
model.fit(
    train_x,train_y,
    validation_split=0.1,
    epochs=50
          )

Epoch 1/50
 252/1125 [=====>........................] - ETA: 13:00 - loss: 0.5581 - acc: 0.6886

KeyboardInterrupt: 

## with XGBOOST

In [8]:
# feature ekstraction with 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
data_features = vectorizer.fit_transform(data['Processed_Reviews'])
data_features = data_features.toarray()

data_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, data['sentiment'], test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


xgb_model = XGBClassifier(max_depth=6, max_delta_step=5).fit(X_train,y_train)
predict = xgb_model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predict)
print(accuracy)

In [38]:
# import xgboost
from xgboost import XGBClassifier

estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [39]:
# xgboost parameter 
params = {
        'min_child_weight': [1, 5, 10]
        # 'gamma': [0.5, 1, 1.5, 2, 5],
        # 'subsample': [0.6, 0.8, 1.0],
        # 'colsample_bytree': [0.6, 0.8, 1.0],
        # 'max_depth': [3, 4, 5]
        }

In [40]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [41]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


c:\Users\alam\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, nthread=4,
                                     num_parallel_tree=None, ...),
             n_jobs=10, param_grid={'min_child_weight': [1, 5, 10]},
             scoring='roc_auc', verbose=True)

In [43]:
grid_search.best_estimator_.min_child_weight

10

In [44]:
from sklearn.metrics import accuracy_score, classification_report
predict = grid_search.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predict)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predict))

Accuracy: 0.862

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      4961
           1       0.85      0.88      0.87      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

